In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

open_ai_model = "gpt-5-nano-2025-08-07"

Enter Your OpenAI API Key:··········


# 🛠️ **Custom Prompt Templates in LangChain:**

- ✂️ **Tailor-Made**: Craft prompts that fit your model like a glove.

- 🎨**Full Control**: Dictate every detail from instructions to formatting.

- 🔌 **Dynamic Inputs**: Plug in specific data as needed for each task.

Create your own by defining inputs and crafting a `format()` method. 📝

📐 Custom templates are like a bespoke suit—made to fit your model's unique requirements.


#🎨 **Custom vs. Default Prompt Templates:**

- ✏️ **Default Templates**: Like using a template for a letter, where you just fill in the blanks.

- 🎭 **Custom Templates**: Like writing a script for a play, where you have the freedom to craft the entire scene.


In [3]:
from langchain_core.prompts import PromptTemplate, StringPromptTemplate

template = PromptTemplate.from_template("Hello {name}!")

prompt = template.format(name="Mohammed")

print(prompt)

Hello Mohammed!



**Custom templates let you:**

- 🛠️ **Programmatically Craft Prompts**: Build prompts on-the-fly, tailored to the task at hand.

- **Example - `FunctionExplainerPromptTemplate`**:

  - 👨🏽‍💻 **Function Input**: Takes a function directly.

  - 🔍 **Code Inspection**: Uses `inspect` to get the function's source code.

  - 🗣️ **Prompt Assembly**: Creates a prompt to explain the function in plain language.

🗝️ This approach is like having a Swiss Army knife for prompt creation, giving you the tools to construct exactly what you need.


In [4]:
import inspect
from langchain_core.prompts import StringPromptTemplate
from pydantic import BaseModel, field_validator

def get_source_code(function_name):
    """Return the source code of the provided function."""
    # Using the inspect module to get the source code of the function
    return inspect.getsource(function_name)

# Template string for the prompt that will be sent to the language model
PROMPT = """Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}

Source Code:
{source_code}

Explanation:
"""

class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input and formats the prompt template to provide the source code of the function."""

    @field_validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        # Ensuring that the only input variable is 'function_name'
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        """Format the prompt using the function's name and source code."""
        # Retrieve the source code of the provided function
        source_code = get_source_code(kwargs["function_name"])

        # Format the PROMPT string using the function name and its source code
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__,
            source_code=source_code
        )
        return prompt

    def _prompt_type(self) -> str:
        """Return the type of prompt."""
        return "function-explainer"

In [5]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])


In [6]:
type(fn_explainer)

__main__.FunctionExplainerPromptTemplate

In [7]:
# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)

print(prompt)

Given the function name and source code, generate an English language explanation of the function.
Function Name: get_source_code

Source Code:
def get_source_code(function_name):
    """Return the source code of the provided function."""
    # Using the inspect module to get the source code of the function
    return inspect.getsource(function_name)


Explanation:



In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=open_ai_model)

response = llm.invoke(prompt)

response.content

'Here is what the function does in plain English:\n\n- Purpose: It returns the Python source code of the function you provide.\n\n- How it works: It uses the inspect.getsource function to fetch the source text for the given function object and returns that text as a string.\n\n- Input expectations: The parameter is named function_name, but you should pass the actual function object (not a string with the function’s name). For example, pass a function like some_function, not "some_function".\n\n- Output: A string containing the source code of the provided function, including its definition and body.\n\n- Limitations and caveats:\n  - If you pass something that inspect.getsource can’t handle (e.g., a string or a built-in function), this will raise an exception.\n  - The function’s source must be available in the current environment (the function must have an accessible source file). Functions defined in an interactive session, dynamically created, or built-in/C-implemented functions may 

In [9]:
from langchain_core.output_parsers import StrOutputParser

llm_chaim = fn_explainer | llm | StrOutputParser()

for chunk in llm_chaim.stream({"function_name":get_source_code}):
  print(chunk, end="", flush=True)

Explanation:

- Purpose: get_source_code returns the textual Python source code of a function that you pass to it.

- How it works: It uses Python’s inspect.getsource to fetch the source code for the provided function object and returns that code as a string.

- Input: function_name is intended to be a function object (i.e., the actual function, not its name as a string). The parameter name is a bit misleading; it should be a callable whose source you want to retrieve.

- Output: A string containing the complete source code of the function, including its def line, body, and any docstring.

- Behavior and limitations:
  - If the argument is not a function (or another object for which source isn’t available), inspect.getsource will raise an exception.
  - It may raise TypeError if you pass something that isn’t a suitable object, or OSError if the source code cannot be found (e.g., if the function is built-in, defined in a dynamically generated context, or not available in a source file).

# Another example


In [10]:
# Template string for the algorithm optimization prompt
ALGO_PROMPT = """Given the algorithm below, suggest optimizations or potential \
improvements, and return the optimized code.

Algorithm Name: {algorithm_name}

Source Code:
{source_code}

Suggestions:
"""

class AlgorithmOptimizerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes an algorithm as input and formats the prompt template to request optimization suggestions."""

    @field_validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "algorithm_function" not in v:
            raise ValueError("algorithm_function must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        """Format the prompt using the algorithm's name and source code."""

        # Retrieve the source code of the provided algorithm
        source_code = get_source_code(kwargs["algorithm_function"])

        # Format the ALGO_PROMPT string using the algorithm name and its source code
        prompt = ALGO_PROMPT.format(
            algorithm_name=kwargs["algorithm_function"].__name__,
            source_code=source_code
        )
        return prompt

    def _prompt_type(self) -> str:
        """Return the type of prompt."""
        return "algorithm-optimizer"


In [11]:
def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion."""
    if n == 0:
        return 1
    else:
        return n * recursive_factorial(n-1)


In [14]:
# Instantiate the AlgorithmOptimizerPromptTemplate with the appropriate input variable
algo_optimizer = AlgorithmOptimizerPromptTemplate(input_variables=["algorithm_function"])

# Generate a prompt for the function "recursive_factorial"
prompt = algo_optimizer.format(algorithm_function=recursive_factorial)

print(prompt)

Given the algorithm below, suggest optimizations or potential improvements, and return the optimized code.

Algorithm Name: recursive_factorial

Source Code:
def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion."""
    if n == 0:
        return 1
    else:
        return n * recursive_factorial(n-1)


Suggestions:



In [15]:
result = llm.invoke(prompt)

print(result.content)

Here are practical optimizations and improvements for factorial computation in Python, followed by optimized code you can use.

Key improvements:
- Avoid Python recursion: Python has a recursion limit (~1000 by default); large n will crash or be slow.
- Validate input: factorial is defined for non-negative integers only; guard against negatives and non-integers.
- Use iterative approach (fast in Python) or the built-in optimized C implementation (math.factorial) for maximum performance.
- If you need to preserve the original function name, provide a wrapper or rename to reflect the implementation.

Option 1: Iterative implementation with input validation (pure Python)
def factorial(n: int) -> int:
    """Compute n! for non-negative integers using an iterative approach.

    Raises:
        TypeError: If n is not an int.
        ValueError: If n < 0.
    """
    if not isinstance(n, int):
        raise TypeError("n must be a non-negative integer")
    if n < 0:
        raise ValueError(

In [16]:
llm_chain = algo_optimizer | llm | StrOutputParser()

for chunk in llm_chain.stream({"algorithm_function":recursive_factorial}):
  print(chunk, end="", flush=True)

Suggestions:
- Validate input: n should be a non-negative integer. Raise TypeError for non-integers and ValueError for negatives.
- Avoid recursion for large n: Python recursion depth limits can cause RecursionError and has overhead. An iterative approach is usually faster and safer.
- Use Python’s built-in math.factorial for optimal performance and correctness.
- If you need a recursive variant for learning, use a tail-recursive style with an accumulator (though Python won’t optimize tail calls; depth will still grow).
- For repeated calls with the same n, a cached recursive version can help, though it’s usually unnecessary for factorial.

Optimized code variants:

1) Iterative implementation (robust and fast, avoids recursion depth issues)
def factorial_iter(n: int) -> int:
    """Compute n! iteratively to avoid recursion depth issues."""
    if not isinstance(n, int):
        raise TypeError("n must be an integer")
    if n < 0:
        raise ValueError("n must be non-negative")
   